In [1]:
import onnx
from onnx.shape_inference import infer_shapes
from onnx import helper, TensorProto
import os
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.core.datatype import DataType
from qonnx.custom_op.general.im2col import compute_conv_output_dim
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.util.basic import gen_finn_dt_tensor

print("ONNX version:", onnx.__version__)

if not os.path.exists("onnx_models"):
    os.mkdir("onnx_models")

ONNX version: 1.11.0


In [3]:
kernel_size, stride, pad = 1,2,0

depthwise = False
in_feature_dim = 7
in_chn = 16

idt = DataType["UINT8"]

if depthwise is True:
    group = out_chn = in_chn
    conv_param_shape = [out_chn, 1, kernel_size, kernel_size]
else:
    group = 1
    out_chn = 20
    conv_param_shape = [out_chn, in_chn, kernel_size, kernel_size]

total_pad = 2 * pad
out_feature_dim = compute_conv_output_dim(
    in_feature_dim, kernel_size, stride, total_pad
)

input_shape = [1, in_chn, in_feature_dim, in_feature_dim]
output_shape = [1, out_chn, out_feature_dim, out_feature_dim]

conv_weight_dt = DataType["UINT4"]

conv_config = {}
conv_config["dilations"] = [1, 1]
conv_config["group"] = group
conv_config["kernel_shape"] = [kernel_size, kernel_size]
conv_config["pads"] = [pad, pad, pad, pad]
conv_config["strides"] = [stride, stride]

top_in = helper.make_tensor_value_info("top_in", TensorProto.FLOAT, input_shape)
top_out = helper.make_tensor_value_info("top_out", TensorProto.FLOAT, output_shape)
value_info = [
    helper.make_tensor_value_info("p1", TensorProto.FLOAT, conv_param_shape)
]

modelproto = helper.make_model(
    helper.make_graph(
        name="conv_test",
        inputs=[top_in],
        outputs=[top_out],
        value_info=value_info,
        nodes=[
            helper.make_node("Conv", ["top_in", "p1"], ["top_out"], **conv_config)
        ],
    )
)

model = ModelWrapper(modelproto)
model.set_tensor_datatype("top_in", idt)
model.set_tensor_datatype("top_out", idt)
model.set_tensor_datatype("p1", conv_weight_dt)
model.set_initializer("p1", gen_finn_dt_tensor(conv_weight_dt, conv_param_shape))

model.save("onnx_models/conv_model.onnx")

In [4]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = "onnx_models/conv_model.onnx"

final_output_dir = "../validate/conv_output"

#Delete previous run results if exist
if os.path.exists(final_output_dir):
    shutil.rmtree(final_output_dir)
    print("Previous run results deleted!")

cfg = build.DataflowBuildConfig(
    output_dir          = final_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    board               = "KV260_SOM",
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)
os.environ["FINN_BUILD_DIR"] = '/workspace/results/single_conv_model/'

In [ ]:
%%time
build.build_dataflow_cfg(model_file, cfg)

Building dataflow accelerator from onnx_models/conv_model.onnx
Intermediate outputs will be generated in /workspace/results/single_conv_model/
Final outputs will be generated in ../validate/conv_output
Build log is at ../validate/conv_output/build_dataflow.log
Running step: step_qonnx_to_finn [1/17]
Running step: step_tidy_up [2/17]
Running step: step_streamline [3/17]
Running step: step_convert_to_hls [4/17]
Running step: step_create_dataflow_partition [5/17]
Running step: step_target_fps_parallelization [6/17]
Running step: step_apply_folding_config [7/17]
Running step: step_generate_estimate_reports [8/17]
Running step: step_hls_codegen [9/17]
Running step: step_hls_ipgen [10/17]
Running step: step_set_fifo_depths [11/17]
Running step: step_create_stitched_ip [12/17]
Running step: step_measure_rtlsim_performance [13/17]
Running step: step_out_of_context_synthesis [14/17]
Running step: step_synthesize_bitfile [15/17]
